In [2]:
from __future__ import annotations

import re
from collections import Counter, defaultdict
from dataclasses import dataclass
from functools import lru_cache
from typing import Dict, List, NamedTuple, Tuple, Union

import black
import jupyter_black

jupyter_black.load(lab=True, target_version=black.TargetVersion.PY310)

# From https://github.com/norvig/pytudes/blob/main/ipynb/Advent-2020.ipynb
def data(day: int, parser=str, sep="\n") -> list:
    "Split the day's input file into sections separated by `sep`, and apply `parser` to each."
    sections = open(f"2021/input-{day}.txt").read().rstrip().split(sep)
    return [parser(section) for section in sections]


def example(day: int, parser=str, sep="\n") -> list:
    "Split the day's example file into sections separated by `sep`, and apply `parser` to each."
    sections = open(f"2021/example-{day}.txt").read().rstrip().split(sep)
    return [parser(section) for section in sections]


def parse_ints(text):
    "Return positive and negative integers found."
    return [int(x) for x in re.findall(r"-?\d+", text)]


def quantify(iterable, pred=bool) -> int:
    "Count the number of items in iterable for which pred is true."
    return sum(1 for item in iterable if pred(item))


Save example data from web site to `example-x.txt` and input data to `input-x.txt` where
`x` is today's date.

In [31]:
# Day 1: Sonar Sweep
depths = data(1, int)

print("Part 1:", sum([new > old for old, new in zip(depths, depths[1:])]))
# Part 2. Given [a, b, c, d] (a+b+c < b+c+d) can be simplified to (a < d)
print("Part 2:", sum([new > old for old, new in zip(depths, depths[3:])]))

Part 1: 1696
Part 2: 1737


In [32]:
# Day 2: Dive!
moves = data(2)

# Part 1: 1990000
horizontal = 0
depth = 0
for move in moves:
    direction, steps = move.split()
    if direction == "forward":
        horizontal += int(steps)
    elif direction == "down":
        depth += int(steps)
    elif direction == "up":
        depth -= int(steps)

print("Part 1:", horizontal * depth)

# Part 2: 1975421260
horizontal = 0
depth = 0
aim = 0
for move in moves:
    direction, steps = move.split()
    if direction == "forward":
        horizontal += int(steps)
        depth += aim * int(steps)
    elif direction == "down":
        aim += int(steps)
    elif direction == "up":
        aim -= int(steps)

print("Part 2:", horizontal * depth)

Part 1: 1990000
Part 2: 1975421260


In [33]:
# Day 2, alternative
class Move(NamedTuple):
    direction: str
    steps: int

    @staticmethod
    def from_line(line: str) -> Move:
        direction, steps = line.split()
        return Move(direction, int(steps))


@dataclass
class Position:
    h: int = 0
    depth: int = 0
    aim: int = 0


# Part 1 and 2 (aim value in part 2 <=> depth value in part 1)
def determine_position(moves) -> Position:
    position = Position()
    for move in moves:
        match move.direction:
            case "forward":
                position.h += move.steps
                position.depth += position.aim * move.steps
            case "down":
                position.aim += move.steps
            case "up":
                position.aim -= move.steps
    return position


# Test with example data
moves = [Move.from_line(line) for line in example(2)]
p = determine_position(moves)
assert p.h == 15
assert p.depth == 60
assert p.aim == 10

# Run with actual input
moves = [Move.from_line(line) for line in data(2)]
p = determine_position(moves)
day2_1 = p.aim * p.h
day2_2 = p.depth * p.h

[day2_1, day2_2]

[1990000, 1975421260]

In [34]:
# Day 3: Binary Diagnostic

values = data(3)
bits = len(values[0])
gamma = ""
for bit_position in zip(*values):
    gamma += "1" if bit_position.count("1") > len(values) / 2 else "0"
gamma = int(gamma, 2)
epsilon = gamma ^ 2 ** bits - 1  # one complement to gamma
print("Part 1:", gamma * epsilon)  # == 4174964

# Day 3, part 2
oxygen = values
for bit in range(bits):
    bit_count = 0
    for value in oxygen:
        bit_count = bit_count + int(value[bit])
    if bit_count >= (len(oxygen) / 2):
        # 1 is most common in position 'bit'
        oxygen = [x for x in oxygen if x[bit] == "1"]
    else:
        oxygen = [x for x in oxygen if x[bit] == "0"]
    if len(oxygen) == 1:
        break

co2 = values
for bit in range(bits):
    bit_count = 0
    for value in co2:
        bit_count += int(value[bit])
    if bit_count >= (len(co2) / 2):
        # 1 is most common in position 'bit'
        co2 = [x for x in co2 if x[bit] == "0"]
    else:
        co2 = [x for x in co2 if x[bit] == "1"]
    if len(co2) == 1:
        break

print("Part 2:", int(oxygen[0], 2) * int(co2[0], 2))  # == 4474944

Part 1: 4174964
Part 2: 4474944


In [35]:
# Day 4: Giant Squid (bingo)
import re


def check_number(board, number):
    "Return row and column if number is in board"
    for row, column in ((r, c) for r in range(5) for c in range(5)):
        if board[row][column] == number:
            return row, column
    return -1, -1


def winning_board(board):
    "Return True if all items in a row or column are true"
    for row in board:
        if sum(row) == -5:
            return True
    for column in zip(*board):
        if sum(column) == -5:
            return True
    return False


def get_score(board):
    "Calculate sum of all values in board where score is 0"
    sum = 0
    for row, column in ((r, c) for r in range(5) for c in range(5)):
        sum = sum + (0 if board[row][column] == -1 else board[row][column])
    return sum


def parse_input(data):
    numbers, *sections = data
    numbers = [int(x) for x in numbers.split(",")]
    sections = [x.split("\n") for x in sections]
    boards = []
    for section in sections:
        board = [parse_ints(row) for row in section]
        boards.append(board)
    return numbers, boards


numbers, boards = parse_input(data(4, sep="\n\n"))

winning_boards, winning_scores = [], []
for number in numbers:
    for i, board in enumerate(boards):
        if i in winning_boards:
            continue
        row, column = check_number(board, number)
        if row > -1:
            # Found number in board
            board[row][column] = -1
            if i not in winning_boards and winning_board(board):
                winning_boards.append(i)
                winning_scores.append(number * get_score(board))

# Part 1, part 2
assert [winning_scores[0], winning_scores[-1]] == [28082, 8224]

In [36]:
# Day 5: Hydrothermal Venture

from collections import Counter


class Point(NamedTuple):
    x: int
    y: int


lines = data(5)
line_coords = [parse_ints(line) for line in lines]
c1 = Counter()
c2 = Counter()

for x1, y1, x2, y2 in line_coords:
    dx = 1 if x1 < x2 else -1
    dy = 1 if y1 < y2 else -1
    if x1 == x2:
        y = y1
        while y != y2 + dy:
            c1[Point(x1, y)] += 1
            y += dy
    elif y1 == y2:
        x = x1
        while x != x2 + dx:
            c1[Point(x, y1)] += 1
            x += dx
    else:
        x = x1
        y = y1
        while x != x2 + dx:
            c2[Point(x, y)] += 1
            x += dx
            y += dy

part1 = sum([v >= 2 for v in c1.values()])
part2 = sum([v >= 2 for v in (c1 + c2).values()])
assert [part1, part2] == [6007, 19349]

In [37]:
# Day 6: Lanternfish
def school_size(fishes: List[int], days: int) -> int:
    "Calculate number of fishes after `days` days"
    fish_counter: Counter = Counter(fishes)
    for _ in range(days):
        births = fish_counter[0]
        for i in range(8):
            fish_counter[i] = fish_counter[i + 1]
        fish_counter[6] += births
        fish_counter[8] = births
    return sum(fish_counter.values())


fishes = example(6, int, ",")
assert [school_size(fishes, 80), school_size(fishes, 256)] == [
    5934,
    26984457539,
]
fishes = data(6, int, ",")
[school_size(fishes, 80), school_size(fishes, 256)]

[383160, 1721148811504]

In [38]:
# Day 7: The Treachery of Whales

from statistics import mean, median


def calculate_fuel(
    positions: List[int], destination: int, cost_function=lambda x: x
) -> int:
    """Amount of fuel to travel from `positions` to `destination` where it costs
    `cost_function` to travel x steps

        Args:
            positions (List[int]): Starting positions
            destination (int): Where to travel
            cost_function ([type], optional): Function to calculate cost

        Returns:
            int: Amount of fuel used
    """
    fuel = 0
    for pos in positions:
        distance = abs(pos - destination)
        fuel += cost_function(distance)
    return fuel


def triangle_number(n: int) -> int:
    """Calculate triangle number (1 + ... + n) for `n`."""
    return (n * (n + 1)) // 2


crabs: List[int] = data(7, int, ",")
part1 = calculate_fuel(crabs, median(crabs))
# mean is often, but not always, correct
# part2 = calculate_fuel(crabs, int(mean(crabs)), triangle_number)
part2 = min(
    calculate_fuel(crabs, x, triangle_number) for x in range(min(crabs), max(crabs) + 1)
)
[part1, part2] == [333755, 94017638]

True

In [39]:
# Day 8: Seven Segment Search
# Decode segments
def decode_simple_numbers(segments: str) -> defaultdict:
    """Returns a dict for numbers 1, 4, 7, 8 where value is a set of chars representing
    that number."""
    display = defaultdict()
    for segment in segments.split():
        segment = set(segment)
        match len(segment):
            case 2:
                display[1] = segment
            case 3:
                display[7] = segment
            case 4:
                display[4] = segment
            case 7:
                display[8] = segment
    return display


def decode_segments(segments: str) -> defaultdict:
    """Returns a display configuration mapping numbers to segments

    Args:
        segments (str): A string with segments separated with spaces

    Returns:
        defaultdict: Keys are numbers 0-9. Values are sets of characters with segments
        used for that number
    """
    display = decode_simple_numbers(segments)
    for segment in segments.split():
        segment = set(segment)
        if len(segment) == 5:
            # Number is 2, 3 or 5
            if len(segment - display[1]) == 3:
                display[3] = segment
            elif len(segment - display[4]) == 2:
                display[5] = segment
            else:
                display[2] = segment
        elif len(segment) == 6:
            # Number is 0, 6 or 9
            if len(segment - display[4]) == 2:
                # 4 matches all segments in 9, but not in 6 or 0
                display[9] = segment
            elif len(segment - display[1]) == 4:
                display[0] = segment
            else:
                display[6] = segment
    return display


lines = data(8)
part1 = 0
part2 = 0
for line in lines:
    result = ""
    segments, values = line.strip().split(" | ")
    display = decode_segments(segments)
    for value in values.split():
        for k, v in display.items():
            if set(value) == v:
                result += str(k)
                if k in [1, 4, 7, 8]:
                    part1 += 1
    part2 += int(result)

assert [part1, part2] == [264, 1063760]

In [40]:
# Day 9: Smoke Basin
lines = data(9)
minimas = []
for y, line in enumerate(lines):
    for x, paren in enumerate(line):
        neighbours = []
        if x > 0 and x < (len(line) - 1):
            neighbours += [line[x - 1], line[x + 1]]
        elif x == 0:
            neighbours += line[x + 1]
        else:
            neighbours += line[x - 1]
        if y > 0 and y < (len(lines) - 1):
            neighbours += [lines[y - 1][x], lines[y + 1][x]]
        elif y == 0:
            neighbours += lines[y + 1][x]
        else:
            neighbours += lines[y - 1][x]
        if paren == min([paren] + neighbours) and int(paren) < 9:
            minimas += paren
print("Part 1:", sum(int(x) + 1 for x in minimas))

# Part 2, find area sizes. This was *hard* for me
# Connected components labeling. See https://en.wikipedia.org/wiki/Connected-component_labeling
lines = data(9)

# Pad image on first row and to the left to avoid bounds checking
labels = [[0] * (len(lines[0]) + 1)]
image = [[0] * (len(lines[0]) + 1)]
for line in lines:
    image.append([0] + [(0 if int(x) == 9 else 1) for x in line])
    labels.append([0] * (len(line) + 1))

next_label = 1
linked = defaultdict(set)
# First pass
for y, line in enumerate(image):
    if y == 0:
        continue
    for x, value in enumerate(line):
        if x == 0:
            continue
        # Set labels
        if value:
            neighbour_labels = []
            if image[y - 1][x]:
                # North label
                neighbour_labels.append(labels[y - 1][x])
            if image[y][x - 1]:
                # West label:
                neighbour_labels.append(labels[y][x - 1])
            if neighbour_labels:
                labels[y][x] = min(neighbour_labels)
                # Update linked, should be a disjointed set
                for label in neighbour_labels:
                    linked[label] = linked[label].union(neighbour_labels)
            else:
                linked[next_label] = {next_label}
                labels[y][x] = next_label
                next_label += 1
# Merge all sets that are not disjoint (i.e. have at least one element in common)
for start in range(len(linked) + 1):
    for end in range(len(linked)):
        if linked[start].isdisjoint(linked[end]):
            pass
        else:
            linked[start] = linked[start].union(linked[end])

# Second pass
for y, line in enumerate(image):
    if y == 0:
        continue
    for x, value in enumerate(line):
        if x == 0:
            continue
        if value:
            labels[y][x] = min(linked[labels[y][x]])

# All areas have same label
c = Counter()
for row in labels:
    c += Counter(row)
part2 = 1
# Skip most common value (0)
for _, val in c.most_common(4)[1:]:
    part2 = part2 * val
print("Part 2:", part2)

Part 1: 580
Part 2: 856716


In [41]:
# Day 9, part 1 and 2 with points - done after submitting answers
# Lots of inspiration from https://www.reddit.com/r/adventofcode/comments/rca6vp/2021_day_9_solutions/hnumswo/


def neighbors(point, grid):
    "Return a list of neighbors to (x,y) that exists in grid"
    (x, y) = point
    candidates = [(x, y - 1), (x, y + 1), (x - 1, y), (x + 1, y)]
    return filter(lambda n: n in grid, candidates)
    # return [p for p in candidates if p in grid] # Does not work since we delete items in
    # grid


def is_low(point, grid):
    "`point` is lower than all neighbors"
    return all(grid[point] < grid[n] for n in neighbors(point, grid))


def count_basin(point, grid):
    if grid[point] == 9:
        return 0
    del grid[point]
    return 1 + sum([count_basin(n, grid) for n in neighbors(point, grid)])


lines = data(9)
points = {
    (x, y): int(height) for y, line in enumerate(lines) for x, height in enumerate(line)
}

minimas = [p for p in points if is_low(p, points)]
part1 = sum(points[x] + 1 for x in minimas)

basins = [count_basin(p, points) for p in minimas]
basins.sort(reverse=True)
part2 = 1
for size in basins[:3]:
    part2 = part2 * size

assert [part1, part2] == [580, 856716]

In [42]:
# Day 10: Syntax Scoring


class CorruptLine(Exception):
    "Raised when a corrupt line is found during parsing"
    pass


def corruption_score(corrupt_parens: str) -> int:
    "Calculate corrupted score. Input is string of first corrupt char in all corrupt lines."
    points = {")": 3, "]": 57, "}": 1197, ">": 25137}
    return sum(Counter(corrupt_parens)[x] * points[x] for x in points)


def completion_score(completion: str) -> int:
    "Calculate score for the string `completion` needed to close an incomplete line."
    points = {")": 1, "]": 2, "}": 3, ">": 4}
    score = 0
    for char in completion:
        score = score * 5 + points[char]
    return score


assert corruption_score("})])>") == 26397
assert completion_score("}}]])})]") == 288957


def parse_line(line) -> str:
    """Parse a line of parenthesis.

    Args:
        line (str): Line to parse

    Raises:
        CorruptLine: Value is first corrupt parenthesis on a corrupt line

    Returns:
        str, str: Corrupt char, all parenthesis needed to complete line
    """
    parens = {"(": ")", "[": "]", "{": "}", "<": ">"}
    needs = ""
    for paren in line:
        if paren in parens.keys():
            # Opening paren
            needs = parens[paren] + needs
        elif paren in parens.values():
            # Closing paren
            if needs[0] == paren:
                # Closing paren matches, remove first char in needs
                needs = needs[1:]
            else:
                raise CorruptLine(paren)
        else:
            # Error in input
            print(f"{paren}: Error in input - should not happen!")
    return needs


corrupt_parens = ""
incomplete_scores = []

lines = data(10)
for line in lines:
    try:
        if needs := parse_line(line):
            incomplete_scores.append(completion_score(needs))
    except CorruptLine as corrupt_paren:
        corrupt_parens += str(corrupt_paren)
        continue

part1 = corruption_score(corrupt_parens)
part2 = sorted(incomplete_scores)[len(incomplete_scores) // 2]
assert [part1, part2] == [399153, 2995077699]

In [43]:
# Day 11: Dumbo Octopus (cavern flashes)
def neighbors(x: int, y: int) -> List[Tuple[int, int]]:
    candidates = [
        (x, y - 1),
        (x, y + 1),
        (x - 1, y),
        (x + 1, y),
        (x - 1, y - 1),
        (x - 1, y + 1),
        (x + 1, y + 1),
        (x + 1, y - 1),
    ]
    return [(x, y) for (x, y) in candidates if (x, y) in energies]


def increase_all_energy_levels():
    for (x, y) in energies:
        energies[x, y] += 1


def flash_point(x: int, y: int):
    flashed[x, y] = True
    for (x_, y_) in neighbors(x, y):
        energies[x_, y_] += 1
    for (x_, y_) in neighbors(x, y):
        if energies[x_, y_] > 9 and not flashed[x_, y_]:
            flash_point(x_, y_)


def flash_all_points():
    for (x, y) in energies:
        if energies[x, y] > 9 and not flashed[x, y]:
            flash_point(x, y)


def reset_flash_status():
    for (x, y) in flashed:
        if flashed[x, y]:
            energies[x, y] = 0
            flashed[x, y] = False


lines = data(11)
energies: Dict[Tuple[int, int], int] = {
    (x, y): int(energy) for y, line in enumerate(lines) for x, energy in enumerate(line)
}
flashed: Dict[Tuple[int, int], bool] = {
    (x, y): False for y, line in enumerate(lines) for x, _ in enumerate(line)
}

flashes_after_100_steps = 0  # Part 1
steps = 0  # Part 2
# Octopuses are in sync when all energies are 0
while (steps < 100) or sum(energies.values()):
    increase_all_energy_levels()
    flash_all_points()
    reset_flash_status()
    if steps < 100:
        # All octopuses with value 0 have flashed
        flashes_after_100_steps += list(energies.values()).count(0)
    steps += 1
print(f"Part 1: {flashes_after_100_steps}, part 2: {steps}")

Part 1: 1625, part 2: 244


In [44]:
# Day 12: Passage Pathing

small = """start-A
start-b
A-c
A-b
b-d
A-end
b-end""".splitlines()

medium = """dc-end
HN-start
start-kj
dc-start
dc-HN
LN-dc
HN-end
kj-sa
kj-HN
kj-dc""".splitlines()

# Part 1
def paths(graph, v):
    path = [v]  # path traversed so far
    seen = {v}  # set of vertices in path

    def search():
        for neighbour in graph[path[-1]]:
            if neighbour not in seen:
                if neighbour.islower():
                    # Never visit this cave again
                    seen.add(neighbour)
                path.append(neighbour)
                yield from search()
                path.pop()
                if neighbour.islower():
                    seen.remove(neighbour)
        if path[-1] == "end":
            yield list(path)

    yield from search()


def paths2(graph, v):
    path = [v]  # path traversed so far

    def search():
        if path[-1] == "end":
            yield list(path)
        for neighbour in graph[path[-1]]:
            lower_case = [x for x in path if x.islower()]
            lower_case_set = set(lower_case)
            blocked = set()
            if len(lower_case) != len(lower_case_set):
                # At least one duplicate, block all small caves
                blocked = lower_case_set
            if neighbour not in blocked:
                path.append(neighbour)
                yield from search()
                path.pop()

    yield from search()


lines = data(12)
caves = defaultdict(list)
for line in lines:
    a, b = line.split("-")
    # Don't return to start, don't leave end
    # Inspiration from https://santé.ti-pun.ch/posts/2021-12-aoc/day12.html
    if b != "start" and a != "end":
        caves[a] += [b]
    if a != "start" and b != "end":
        caves[b] += [a]

print("Part 1:", len(list(paths(caves, "start"))))  # 4495
print("Part 2:", len(list(paths2(caves, "start"))))  # 131254

Part 1: 4495
Part 2: 131254


In [45]:
# Day 13: Transparent Origami


def fold_point(x, y, direction, fold_line):
    if direction == "x" and x > fold_line:
        return (fold_line * 2 - x, y)
    elif direction == "y" and y > fold_line:
        return (x, fold_line * 2 - y)
    else:
        # No fold
        return (x, y)


assert fold_point(2, 13, "y", 7) == (2, 1)
assert fold_point(6, 2, "x", 5) == (4, 2)
assert fold_point(0, 3, "y", 4) == (0, 3)


def fold_points(points, fold):
    direction, fold_line = fold
    return {fold_point(x, y, direction, fold_line) for (x, y) in points}


points_lines, folds_lines = data(13, sep="\n\n")
points = {tuple(parse_ints(line)) for line in points_lines.split()}
folds = [
    (line.split("=")[0][-1], int(line.split("=")[1]))
    for line in folds_lines.splitlines()
]

# Part 1. Number of points after first fold
print(len(fold_points(points, folds[0])))  # 647

# Part 2
result = points
for fold in folds:
    result = fold_points(result, fold)

# Printing apapted from https://www.reddit.com/r/adventofcode/comments/rf7onx/comment/hod6ieo/?utm_source=reddit&utm_medium=web2x&context=3
for y in range(6):
    print(*[["  ", "██"][(x, y) in result] for x in range(40)])

647
██       ██    ██ ██ ██ ██          ██ ██    ██       ██          ██ ██    ██ ██ ██          ██ ██             ██ ██   
██       ██    ██                      ██    ██       ██             ██    ██       ██    ██       ██             ██   
██ ██ ██ ██    ██ ██ ██                ██    ██ ██ ██ ██             ██    ██       ██    ██                      ██   
██       ██    ██                      ██    ██       ██             ██    ██ ██ ██       ██                      ██   
██       ██    ██             ██       ██    ██       ██    ██       ██    ██    ██       ██       ██    ██       ██   
██       ██    ██ ██ ██ ██       ██ ██       ██       ██       ██ ██       ██       ██       ██ ██          ██ ██      


In [46]:
# Day 14: Extended Polymerization
def generate_pair_counter(template: str) -> Counter:
    c = Counter()
    for i in range(len(template) - 1):
        pair = template[i : i + 2]
        c[pair] += 1
    return c


def insert_pairs(pairs: Counter) -> Counter:
    "Insert new pairs based on rules in `replacements`. Returns a Counter with new pairs"
    global replacements

    new_pairs = Counter()
    for (a, b), count in pairs.items():
        new_char = replacements[a + b]
        char_counter[new_char] += count
        new_pairs[a + new_char] += count
        new_pairs[new_char + b] += count

    return new_pairs


template, _, *lines = open("2021/input-14.txt").read().strip().split("\n")
replacements = dict(line.split(" -> ") for line in lines)

# Step 0
char_counter = Counter(template)
pairs = generate_pair_counter(template)

for step in range(10):
    pairs = insert_pairs(pairs)

# Should be 2967
print("Part 1:", char_counter.most_common()[0][1] - char_counter.most_common()[-1][1])

for step in range(30):
    pairs = insert_pairs(pairs)

# 3692219987038
print("Part 2:", char_counter.most_common()[0][1] - char_counter.most_common()[-1][1])

Part 1: 2967
Part 2: 3692219987038


In [47]:
# Day 15: Chiton (Dijsktra with priority queue)
# From https://gist.github.com/qpwo/cda55deee291de31b50d408c1a7c8515

from queue import PriorityQueue  # essentially a binary heap


def dijkstra(G, start, goal):
    """Uniform-cost search / dijkstra"""
    visited = set()
    cost = {start: 0}
    parent = {start: None}
    todo = PriorityQueue()

    todo.put((0, start))
    while todo:
        while not todo.empty():
            _, vertex = todo.get()  # finds lowest cost vertex
            # loop until we get a fresh vertex
            if vertex not in visited:
                break
        else:  # if todo ran out
            break  # quit main loop
        visited.add(vertex)
        if vertex == goal:
            break
        for neighbor, distance in G[vertex].items():
            if neighbor in visited:
                continue  # skip these to save time
            old_cost = cost.get(neighbor, float("inf"))  # default to infinity
            new_cost = cost[vertex] + distance
            if new_cost < old_cost:
                todo.put((new_cost, neighbor))
                cost[neighbor] = new_cost
                parent[neighbor] = vertex

    return cost


def neighbors(point, grid):
    "Return a list of neighbors to (x,y) that exists in grid"
    (x, y) = point
    candidates = [(x, y - 1), (x, y + 1), (x - 1, y), (x + 1, y)]
    return {p for p in candidates if p in grid}


def create_5x_grid(grid: defaultdict) -> defaultdict:
    """
    Create a grid 5 times larger than input. Increase values in grid with 1 each time grid
    is duplicated to x, and y
    """
    grid_size = max(grid)[0] + 1
    large_grid = defaultdict(int)
    for x_ in range(5):
        for y_ in range(5):
            for (x, y) in grid:
                new_cost = grid[x, y] + x_ + y_
                # 10 wraps around to 1, 11 to 2, etc
                new_cost = new_cost % 10 + new_cost // 10
                large_grid[x + x_ * grid_size, y + y_ * grid_size] = new_cost
    return large_grid


lines = data(15)
part1_grid = {
    (x, y): int(cost) for y, line in enumerate(lines) for x, cost in enumerate(line)
}
part2_grid = create_5x_grid(part1_grid)

edges = defaultdict(lambda: defaultdict(int))
for destination in part2_grid:
    for n in neighbors(destination, part2_grid):
        edges[n][destination] = int(part2_grid[destination])


cost = dijkstra(edges, (0, 0), max(edges))
print("Part 1:", cost[max(part1_grid)])
print("Part 2:", cost[max(part2_grid)])

Part 1: 361
Part 2: 2838


In [48]:
# Day 15 Using NetworkX
import networkx as nx


lines = data(15)
part1_grid = {
    (x, y): int(cost) for y, line in enumerate(lines) for x, cost in enumerate(line)
}
part2_grid = create_5x_grid(part1_grid)

edges = defaultdict(lambda: defaultdict(int))
for destination in part2_grid:
    for n in neighbors(destination, part2_grid):
        edges[n][destination] = int(part2_grid[destination])

# Needs to be a DiGraph and not Graph.
# See https://github.com/mebeim/aoc/blob/master/2021/README.md#day-15---chiton
G = nx.DiGraph()
for from_ in edges:
    for to_ in edges[from_]:
        G.add_edge(from_, to_, weight=edges[from_][to_])

print("Part 1:", nx.shortest_path_length(G, (0, 0), max(part1_grid), weight="weight"))
print("Part 2:", nx.shortest_path_length(G, (0, 0), max(part2_grid), weight="weight"))

Part 1: 361
Part 2: 2838


In [49]:
# Day 15 from Joel Grus
from typing import List
from heapq import heappush, heappop

RAW = """1163751742
1381373672
2136511328
3694931569
7463417111
1319128137
1359912421
3125421639
1293138521
2311944581"""

Grid = List[List[int]]


def parse(raw: str) -> Grid:
    return [[int(x) for x in line] for line in raw.splitlines()]


def lowest_cost_path(grid: Grid) -> int:
    number_of_rows = len(grid)
    number_of_cols = len(grid[0])
    iterations = 0

    # Queue with (row, col), cost
    q = []
    # Starting position (0,0)
    heappush(q, (0, 0, 0))
    visited = {(0, 0)}
    while q:
        iterations += 1
        # Cost to reach r, c
        cost, r, c = heappop(q)
        # print("current:", cost, (r, c), "visited:", visited, "queue:", q)
        if r == number_of_rows - 1 and c == number_of_cols - 1:
            return cost, iterations
        for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            r_ = r + dr
            c_ = c + dc
            if (
                0 <= r_ < number_of_rows
                and 0 <= c_ < number_of_cols
                and (r_, c_) not in visited
            ):
                # print("add to queue:", grid[r_][c_], (r_, c_))
                heappush(q, (cost + grid[r_][c_], r_, c_))
                visited.add((r_, c_))
    raise Exception("No path found")


GRID = parse(RAW)
lowest_cost_path(GRID)

grid = parse(open("2021/input-15.txt").read())
lowest_cost_path(grid)

(361, 9996)

In [50]:
# Day 16: Packet Decoder
class Packet(NamedTuple):
    version: int
    type_id: int
    value: int = 0
    subpackets: List[Packet] = []


def hex_to_bin(hex: str) -> str:
    # Does not pad with leading zeroes, but that seems to work for input data for day 16
    # Using an f-string might be better, see https://stackoverflow.com/a/16926357/5821338
    return bin(int(hex, 16))[2:]


assert hex_to_bin("D2FE28") == "110100101111111000101000"
assert (
    hex_to_bin("EE00D40C823060")
    == "11101110000000001101010000001100100000100011000001100000"
)


def decode_value_string(binary: str) -> tuple(int, int):
    """Decodes value in string `binary`. Returns value and the position where next packet starts"""
    # Literal value
    pos = 0
    bin_value = ""
    while int(binary[pos]):
        bin_value += binary[pos + 1 : pos + 5]
        pos += 5
    # Decode last value chunk (starts with 0)
    bin_value += binary[pos + 1 : pos + 5]
    return int(bin_value, 2), pos + 5


assert decode_value_string("101111111000101000") == (2021, 15)


def make_sub_tree(binary: str) -> Tuple[Packet, str]:
    HEADER = 6
    version = int(binary[0:3], 2)  # Bit 0-2
    type_id = int(binary[3:6], 2)  # Bit 3-5
    if type_id == 4:
        # Decode value starting in position 6
        value, next_packet_start = decode_value_string(binary[HEADER:])
        tail = binary[HEADER + next_packet_start :]
        return Packet(version, type_id, value), tail
    return make_packet_with_children(binary, version, type_id)


def make_packet_with_children(binary: str, version, type_id) -> Tuple[Packet, str]:
    HEADER = 6
    length_type_id = binary[HEADER]
    packet = Packet(version, type_id, 0, subpackets=[])
    if length_type_id == "0":
        LENGTH_BITS = 15
        # The next 15 bits are a number that represents the total length in bits of
        # the sub-packets contained by this packet.
        sub_packets_length = int(binary[HEADER + 1 : HEADER + 1 + LENGTH_BITS], 2)
        sub_packets = binary[
            HEADER + 1 + LENGTH_BITS : HEADER + 1 + LENGTH_BITS + sub_packets_length
        ]
        tail = binary[HEADER + 1 + LENGTH_BITS + sub_packets_length :]
        while len(sub_packets) > 0:
            subpacket, sub_packets = make_sub_tree(sub_packets)
            packet.subpackets.append(subpacket)
    else:
        NUMBER_BITS = 11
        # The next 11 bits are a number that represents the number of sub-packets
        # immediately contained by this packet.
        number_of_sub_packets = int(binary[HEADER + 1 : HEADER + 1 + NUMBER_BITS], 2)
        sub_packets = binary[HEADER + 1 + NUMBER_BITS :]
        for i in range(number_of_sub_packets):
            subpacket, sub_packets = make_sub_tree(sub_packets)
            packet.subpackets.append(subpacket)
        tail = sub_packets
    return packet, tail


def get_version_sum(node: Packet) -> int:
    return node.version + sum(get_version_sum(x) for x in node.subpackets)


from math import prod


def calculate(node: Packet):
    if node.type_id == 0:
        return sum(calculate(x) for x in node.subpackets)
    elif node.type_id == 1:
        return prod(calculate(x) for x in node.subpackets)
    elif node.type_id == 2:
        return min(calculate(x) for x in node.subpackets)
    elif node.type_id == 3:
        return max(calculate(x) for x in node.subpackets)
    elif node.type_id == 4:
        return node.value
    elif node.type_id == 5:
        return int(calculate(node.subpackets[0]) > calculate(node.subpackets[1]))
    elif node.type_id == 6:
        return int(calculate(node.subpackets[0]) < calculate(node.subpackets[1]))
    elif node.type_id == 7:
        return int(calculate(node.subpackets[0]) == calculate(node.subpackets[1]))


DATA = "E20D4100AA9C0199CA6A3D9D6352294D47B3AC6A4335FBE3FDD251003873657600B46F8DC600AE80273CCD2D5028B6600AF802B2959524B727D8A8CC3CCEEF3497188C017A005466DAA6FDB3A96D5944C014C006865D5A7255D79926F5E69200A164C1A65E26C867DDE7D7E4794FE72F3100C0159A42952A7008A6A5C189BCD456442E4A0A46008580273ADB3AD1224E600ACD37E802200084C1083F1540010E8D105A371802D3B845A0090E4BD59DE0E52FFC659A5EBE99AC2B7004A3ECC7E58814492C4E2918023379DA96006EC0008545B84B1B00010F8E915E1E20087D3D0E577B1C9A4C93DD233E2ECF65265D800031D97C8ACCCDDE74A64BD4CC284E401444B05F802B3711695C65BCC010A004067D2E7C4208A803F23B139B9470D7333B71240050A20042236C6A834600C4568F5048801098B90B626B00155271573008A4C7A71662848821001093CB4A009C77874200FCE6E7391049EB509FE3E910421924D3006C40198BB11E2A8803B1AE2A4431007A15C6E8F26009E002A725A5292D294FED5500C7170038C00E602A8CC00D60259D008B140201DC00C401B05400E201608804D45003C00393600B94400970020C00F6002127128C0129CDC7B4F46C91A0084E7C6648DC000DC89D341B23B8D95C802D09453A0069263D8219DF680E339003032A6F30F126780002CC333005E8035400042635C578A8200DC198890AA46F394B29C4016A4960C70017D99D7E8AF309CC014FCFDFB0FE0DA490A6F9D490010567A3780549539ED49167BA47338FAAC1F3005255AEC01200043A3E46C84E200CC4E895114C011C0054A522592912C9C8FDE10005D8164026C70066C200C4618BD074401E8C90E23ACDFE5642700A6672D73F285644B237E8CCCCB77738A0801A3CFED364B823334C46303496C940"

root, _ = make_sub_tree(hex_to_bin(DATA))
(get_version_sum(root), calculate(root))

(925, 342997120375)

In [51]:
# Day 17: Trick Shot

# Manually enter target area, don't bother with parsing
# Example: target area: x=20..30, y=-10..-5
# min_x = 20
# max_x = 30
# max_y = -5
# min_y = -10

# My input: target area: x=111..161, y=-154..-101
min_x = 111
max_x = 161
min_y = -154
max_y = -101


def step(x: int, y: int, x_vel: int, y_vel: int) -> Tuple(int, int, int, int):
    """The probe's x,y position starts at 0,0. Then, it will follow some trajectory by moving in steps. On each step, these changes occur in the following order:

    The probe's x position increases by its x velocity.
    The probe's y position increases by its y velocity.
    Due to drag, the probe's x velocity changes by 1 toward the value 0; that is, it decreases by 1 if it is greater than 0, increases by 1 if it is less than 0, or does not change if it is already 0.
    Due to gravity, the probe's y velocity decreases by 1.
    """
    x_ = x + x_vel
    y_ = y + y_vel
    if x_vel > 0:
        x_vel_ = x_vel - 1
    elif x_vel < 0:
        x_vel_ = x_vel + 1
    else:
        x_vel_ = x_vel
    y_vel_ = y_vel - 1
    return (x_, y_, x_vel_, y_vel_)


def hits_target(x_vel: int, y_vel: int) -> Tuple[bool, int, int, int]:
    "Does this starting velocity hit the target, returns x,y position of last step and highest y reached"

    def inside_target(x: int, y: int) -> bool:
        "Returns True if x and y is inside target"
        return (min_y <= y <= max_y) and (min_x <= x <= max_x)

    x, y = 0, 0
    max_found = y
    while not inside_target(x, y):
        x, y, x_vel, y_vel = step(x, y, x_vel, y_vel)
        max_found = y if y > max_found else max_found
        # If x or y has passed target, we will never hit it
        if x > max_x or y < min_y:
            return False, x, y, max_found
    return True, x, y, max_found


# If you have an x velocity larger than the max x coordinate of the target you will
# overshoot after the first step. This means you only need to check from 1 to max_x
# (including max_x).
#
# In the same way a y velocity lower than min_y of your target will miss after one step.
# Lower bound to try is min_y. If you shoot something up with velocity y it will come down
# to the surface with velocity -y. Having a higher velocity than the lowest y coordinate
# will miss the target. Upper bound for y velocity is then -max_y.

y_max = 0
hits = []
for x_vel in range(1, max_x + 1):
    for y_vel in range(min_y, -min_y):
        did_hit, x, y, y_max_ = hits_target(x_vel, y_vel)
        if did_hit:
            if y_max_ > y_max:
                y_max = y_max_
            hits.append((x_vel, y_vel))


y_max, len(hits)

(11781, 4531)

In [52]:
# Day 18: Snailfish

import ast
import math

# Binary tree. Left is 0, right is 1. Index is a string representing the binary number
# Value of -1 indicates there are children, otherwise value.
# len(index) is depth of node in tree
def make_pairs(pairs: defaultdict, pair: list, index=""):
    left_index = index + "0"
    right_index = index + "1"
    if isinstance(pair[0], int):
        pairs[left_index] = pair[0]
    else:
        pairs[left_index] = -1
        make_pairs(pairs, pair[0], left_index)

    if isinstance(pair[1], int):
        pairs[right_index] = pair[1]
    else:
        pairs[right_index] = -1
        make_pairs(pairs, pair[1], right_index)
    return pairs


def explode_pair(pairs, index):
    """
    To explode a pair, the pair's left value is added to the first regular number to the
    left of the exploding pair (if any), and the pair's right value is added to the first
    regular number to the right of the exploding pair (if any). Exploding pairs will
    always consist of two regular numbers. Then, the entire exploding pair is replaced
    with the regular number 0.
    """
    left_index = index + "0"
    right_index = index + "1"
    # print(f"Explode {pairs[left_index], pairs[right_index]}, index {index}")
    regular_number_indices = [k for k, v in pairs.items() if v >= 0]
    regular_number_indices.sort()
    if regular_number_indices.index(left_index) == 0:
        # No left neighbor
        pass
    else:
        left_neighbor = regular_number_indices[
            regular_number_indices.index(left_index) - 1
        ]
        pairs[left_neighbor] += pairs[left_index]

    if regular_number_indices.index(right_index) == len(regular_number_indices) - 1:
        # No right neighbor
        pass
    else:
        right_neighbor = regular_number_indices[
            regular_number_indices.index(right_index) + 1
        ]
        pairs[right_neighbor] += pairs[right_index]
    del pairs[left_index]
    del pairs[right_index]
    pairs[index] = 0


def split_number(pairs, index):
    """
    To split a regular number, replace it with a pair; the left element of the pair should
    be the regular number divided by two and rounded down, while the right element of the
    pair should be the regular number divided by two and rounded up. For example, 10
    becomes [5,5], 11 becomes [5,6], 12 becomes [6,6], and so on.

    During reduction, at most one action applies, after which the process returns to the top of the list of actions. For example, if split produces a pair that meets the explode criteria, that pair explodes before other splits occur.
    """
    left_index = index + "0"
    right_index = index + "1"
    pairs[left_index] = math.floor(pairs[index] / 2)
    pairs[right_index] = math.ceil(pairs[index] / 2)
    pairs[index] = -1
    if len(index) == 4:
        explode_pair(pairs, index)


def needs_exploding(pairs: defaultdict) -> str:
    "Returns the index of first pair that needs to explode"
    for k, v in sorted(pairs.items()):
        if len(k) == 4 and v == -1:
            return k
    return False


def needs_splitting(pairs: defaultdict) -> str:
    for k, v in sorted(pairs.items()):
        if v >= 10:
            return k
    return False


def add_pairs(left_pairs, right_pairs) -> defaultdict:
    result = defaultdict(lambda: -1)
    for k, v in left_pairs.items():
        left_index = "0" + k
        result[left_index] = v
        # Set parent (i.e. all everyting but last bit of index) to -1
        result[left_index[:-1]] = -1
    for k, v in right_pairs.items():
        right_index = "1" + k
        result[right_index] = v
        result[right_index[:-1]] = -1
    return result


def print_pairs(pairs):
    depth = 0
    for k, v in sorted(pairs.items()):
        if len(k) > depth:
            print("".join("[" * (len(k) - depth)), end="")
            depth = len(k)
        elif len(k) < depth:
            print("".join("]" * (depth - len(k))) + ", ", end="")
        elif len(k) == depth:
            print(", ", end="")
        depth = len(k)
        if v > -1:
            print(v, end="")
    print()


def calculate_magnitude(pairs: defaultdict, index="") -> int:
    """
    To check whether it's the right answer, the snailfish teacher only checks the
    magnitude of the final sum. The magnitude of a pair is 3 times the magnitude of its
    left element plus 2 times the magnitude of its right element. The magnitude of a
    regular number is just that number.

    For example, the magnitude of [9,1] is 3*9 + 2*1 = 29; the magnitude of [1,9] is 3*1 +
    2*9 = 21. Magnitude calculations are recursive: the magnitude of [[9,1],[1,9]] is 3*29
    + 2*21 = 129.
    """
    left_index = index + "0"
    right_index = index + "1"
    if pairs[left_index] != -1:
        left_magnitude = pairs[left_index]
    else:
        left_magnitude = calculate_magnitude(pairs, left_index)
    if pairs[right_index] != -1:
        right_magnitude = pairs[right_index]
    else:
        right_magnitude = calculate_magnitude(pairs, right_index)
    return 3 * left_magnitude + 2 * right_magnitude


# Part 1
pairs_to_add = data(18, parser=ast.literal_eval)
pairs = defaultdict(lambda: -1)
make_pairs(pairs, pairs_to_add.pop(0))
for pair_to_add in pairs_to_add:
    right_pairs = defaultdict(lambda: -1)
    make_pairs(right_pairs, pair_to_add)
    pairs = add_pairs(pairs, right_pairs)
    # Explode and split
    while index := needs_exploding(pairs):
        explode_pair(pairs, index)
    while split_index := needs_splitting(pairs):
        split_number(pairs, split_index)
# Part 1 == 4235
assert calculate_magnitude(pairs, "") == 4235

# Begin part 2
pairs_to_add = data(18, parser=ast.literal_eval)
magnitudes = []
for a in range(len(pairs_to_add) - 1):
    for b in range(1, len(pairs_to_add)):
        if a == b:
            continue
        pairs_a = defaultdict(lambda: -1)
        pairs_b = defaultdict(lambda: -1)
        make_pairs(pairs_a, pairs_to_add[a])
        make_pairs(pairs_b, pairs_to_add[b])
        pairs = add_pairs(pairs_a, pairs_b)
        # Explode and split
        while index := needs_exploding(pairs):
            explode_pair(pairs, index)
        while split_index := needs_splitting(pairs):
            split_number(pairs, split_index)
        magnitudes.append(calculate_magnitude(pairs))
        pairs = add_pairs(pairs_b, pairs_a)
        # Explode and split
        while index := needs_exploding(pairs):
            explode_pair(pairs, index)
        while split_index := needs_splitting(pairs):
            split_number(pairs, split_index)
        magnitudes.append(calculate_magnitude(pairs))

# Part 2: 4659. Runs in 3.3 s
max(magnitudes)

4659

In [53]:
# Day 19: Beacon Scanner

from itertools import combinations
from math import sqrt


@dataclass
class Scanner:
    id: int
    beacons: List[Tuple(int, int, int)]
    translation: Tuple(int, int, int) = (0, 1, 2)
    sign: Tuple(int, int, int) = (1, 1, 1)
    offset: Tuple(int, int, int) = (0, 0, 0)


def parse_ints(text):
    "Return positive and negative integers found."
    return [int(x) for x in re.findall(r"-?\d+", text)]


def manhattan_distance(p1, p2):
    return sum(abs(p1[i] - p2[i]) for i in range(3))


def distance(x1, y1, z1, x2, y2, z2):
    "Euclidian distance between two points in 3D space"
    return sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2 + (z1 - z2) ** 2)


def get_distances(beacons):
    "Return a dict with distances between all points in list `beacons`"
    return {distance(*p1, *p2): (p1, p2) for p1, p2 in combinations(beacons, 2)}


def get_translation(a1, a2, b1, b2):
    "Return translation between two scanners' coordinate systems. Beacons a1 and a2 as seen by one scanner and b1, b2 seen by another."
    dx, dy, dz = tuple(a1[i] - a2[i] for i in range(3))
    _dx, _dy, _dz = tuple(b1[i] - b2[i] for i in range(3))
    sign = [1, 1, 1]
    translation = [0, 1, 2]
    # See how X coordinates in scanner 1 translates to scanner 0:s coordinate system
    if abs(dx) == abs(_dy):
        translation[0] = 1
        sign[0] = dx // _dy
    elif abs(dx) == abs(_dz):
        translation[0] = 2
        sign[0] = dx // _dz
    else:
        assert abs(dx) == abs(_dx)
        sign[0] = dx // _dx
    # Y
    if abs(dy) == abs(_dx):
        translation[1] = 0
        sign[1] = dy // _dx
    elif abs(dy) == abs(_dz):
        translation[1] = 2
        sign[1] = dy // _dz
    else:
        assert abs(dy) == abs(_dy)
        sign[1] = dy // _dy
    # Z
    if abs(dz) == abs(_dx):
        translation[2] = 0
        sign[2] = dz // _dx
    elif abs(dz) == abs(_dy):
        translation[2] = 1
        sign[2] = dz // _dy
    else:
        assert abs(dz) == abs(_dz)
        sign[2] = dz // _dz

    return translation, sign


def difference(a, b):
    "Calculate difference between coordinates a and b"
    return a[0] - b[0], a[1] - b[1], a[2] - b[2]


def translate_coord(coord, translation, sign):
    x, y, z = (coord[translation[i]] * sign[i] for i in range(3))
    return x, y, z


def offset_coord(coord, offset):
    x, y, z = (coord[i] + offset[i] for i in range(3))
    return x, y, z


def set_coordinate_system(a: Scanner, b: Scanner):
    "Given scanners a and b, set translation, sign, and offset in scanner b to match scanner a"

    def validate_candidate(a, b1, b2, translation, sign, offset):
        "Return true if b1 or b2 transposes to a given `translation`, `sign`, and `offset`."
        b1 = translate_coord(b1, translation, sign)
        b1 = offset_coord(b1, offset)
        b2 = translate_coord(b2, translation, sign)
        b2 = offset_coord(b2, offset)
        if a in (b1, b2):
            return True
        return False

    distances_a = get_distances(a.beacons)
    distances_b = get_distances(b.beacons)
    common_distances = [d for d in distances_a if d in distances_b]

    pair_index = common_distances[0]
    a1, a2 = distances_a[pair_index]
    b1, b2 = distances_b[pair_index]
    # Check result with two other points
    pair_index = common_distances[1]
    check_a, _ = distances_a[pair_index]
    check_b1, check_b2 = distances_b[pair_index]

    # Need to try both orders of pairs in b
    poss_trans, poss_sign = get_translation(a1, a2, b1, b2)
    _b1 = translate_coord(b1, poss_trans, poss_sign)
    poss_offset = difference(a1, _b1)
    if not validate_candidate(
        check_a, check_b1, check_b2, poss_trans, poss_sign, poss_offset
    ):
        # Pairs in b must be in other order
        poss_trans, poss_sign = get_translation(a1, a2, b2, b1)
        _b2 = translate_coord(b2, poss_trans, poss_sign)
        poss_offset = difference(a1, _b2)

    b.translation = poss_trans
    b.sign = poss_sign
    b.offset = poss_offset


def update_all_beacon_coordinates(scanner: Scanner):
    "Update all beacon coordinates according to scanners translation, sign, and offset"
    for i, beacon in enumerate(scanner.beacons):
        coord = translate_coord(beacon, scanner.translation, scanner.sign)
        coord = offset_coord(coord, scanner.offset)
        scanner.beacons[i] = coord


def overlaps(a: Scanner, b: Scanner) -> bool:
    "Returns True if 12 or more beacons overlap between scanner a and scanner b"
    dist_a = get_distances(a.beacons)
    dist_b = get_distances(b.beacons)
    common_distances = [d for d in dist_a if d in dist_b]
    return len(common_distances) >= 66


scanners = []
chunks = data(19, sep="\n\n")
for chunk in chunks:
    id, *beacons = chunk.splitlines()
    id = int(id.split()[2])
    coords = [tuple(parse_ints(beacon)) for beacon in beacons]
    scanners.append(Scanner(id=id, beacons=coords))

scanners_to_align = scanners.copy()
scanners_to_check_against = [scanners_to_align.pop(0)]
matched_beacons = set(scanners_to_check_against[0].beacons)
while scanners_to_align:
    a = scanners_to_check_against.pop()
    for b in scanners_to_align.copy():
        if overlaps(a, b):
            set_coordinate_system(a, b)
            update_all_beacon_coordinates(b)
            matched_beacons = matched_beacons.union(set(b.beacons))
            scanners_to_align.remove(b)
            scanners_to_check_against.append(b)

print("Part 1:", len(matched_beacons))  # 362
print(
    "Part 2:",
    max(
        manhattan_distance(s1.offset, s2.offset) for s1, s2 in combinations(scanners, 2)
    ),
)  # 12204

Part 1: 362
Part 2: 12204


In [54]:
# Day 20: Trench Map
def extend_image(image, size):
    "Extend image in all directions with 'size'"
    input_image = []
    for _ in range(size):
        input_image += ["." * (len(image) + size * 2)]

    input_image += [("." * size) + row + ("." * size) for row in image]
    input_image += ["." * (len(image) + size * 2)]
    for _ in range(size - 1):
        input_image += ["." * (len(image) + size * 2)]
    return input_image


def neighbors(grid, r, c):
    "Return a 3x3 matrix surrounding position row, column. Assumes r and c is inside of grid."
    rows = grid[r - 1 : r + 2]
    return [row[c - 1 : c + 2] for row in rows]


def to_index(grid):
    "Converts a 3x3 matrix to a number. ['...', '#..','.#.']-> 000 100 010 = 34"
    binary = (
        "".join([val for row in grid for val in row])
        .replace(".", "0")
        .replace("#", "1")
    )
    return int(binary, 2)


assert to_index(["...", "#..", ".#."]) == 34


def create_output(image, algorithm):
    """Every pixel of the infinite output image needs to be calculated exactly based on
    the relevant pixels of the input image.

    Since algorithm[0] = '#' and algorithm[511] = '.' this means all pixels will change
    value for each iteration. Test extending input image a bit more to see what happens
    """
    input_image = extend_image(image, 2)
    output_image = []
    for r in range(1, len(input_image) - 1):
        output_row = ""
        for c in range(1, len(input_image) - 1):
            index = to_index(neighbors(input_image, r, c))
            output_row += algorithm[index]
        output_image.append(output_row)
    return output_image


def test_input(day: int, parser=str, sep="\n") -> list:
    "Split the day's example file into sections separated by `sep`, and apply `parser` to each."
    sections = open(f"2021/test-{day}.txt").read().rstrip().split(sep)
    return [parser(section) for section in sections]


def iterate_and_count(image, algorithm, iterations):
    "Perform image enhancement `iterations` times, return number of # in last image."
    size = iterations * 2
    result = extend_image(image, size)
    for i in range(iterations):
        result = create_output(result, algorithm)
        print(i + 1, end=".")  # Takes 15 seconds to complete, show progress

    # Implementation adds a border size large. Remove border before counting
    for _ in range(size):
        result.pop(0)
        result.pop()

    return sum(row[size:-size].count("#") for row in result)


algorithm, lines = data(20, sep="\n\n")
image = lines.splitlines()
print("Part 1:", iterate_and_count(image, algorithm, 2))  # 3351
print("Part 2:", iterate_and_count(image, algorithm, 50))  # 16383

1.2.Part 1: 5391
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.25.26.27.28.29.30.31.32.33.34.35.36.37.38.39.40.41.42.43.44.45.46.47.48.49.50.Part 2: 16383


In [55]:
# Day 21: Dirac Dice


@dataclass
class Dice:
    # 0 value will never be used since we start by rolling the dice
    current: int = 0
    rolls: int = 0

    def roll(self):
        self.current += 1
        if self.current == 101:
            self.current = 1
        self.rolls += 1


@dataclass
class Player:
    pos: int
    score: int = 0

    def move(self, steps: int):
        """
        Move `steps` forward from `pos` on a circular track from 1-10 and update score.
        """
        n = self.pos + steps
        if n // 10 == n / 10:
            self.pos = 10
        else:
            self.pos = n % 10
        self.score += self.pos

    def play_round(self, dice: Dice):
        dice_rolls = []
        for _ in range(3):
            dice.roll()
            dice_rolls.append(dice.current)
        steps = sum(dice_rolls)
        self.move(steps)


# Example input
p1 = Player(pos=4)
p2 = Player(pos=8)

# My input
p1 = Player(pos=4)
p2 = Player(pos=10)

# Part 1
dice = Dice()
print("Part 1:", end=" ")
while True:
    p1.play_round(dice)
    if p1.score >= 1000:
        print(dice.rolls * p2.score)
        break
    p2.play_round(dice)
    if p2.score >= 1000:
        print(dice.rolls * p1.score)
        break

# Day 21 part 2
def move(pos, steps: int) -> int:
    """
    Move `steps` forward from `pos` on a circular track from 1-10. Return new position.
    """
    n = pos + steps
    if n // 10 == n / 10:
        return 10
    else:
        return n % 10


universes_in_result = {3: 1, 4: 3, 5: 6, 6: 7, 7: 6, 8: 3, 9: 1}

# Working
universes_in_result = {3: 1, 4: 3, 5: 6, 6: 7, 7: 6, 8: 3, 9: 1}
universes = {(0, 4, 0, 10): 1}

p1wins = 0
p2wins = 0
while universes:
    nuv = defaultdict(int)
    for state, cnt in universes.items():
        score1, pos1, score2, pos2 = state
        for d, dcount in universes_in_result.items():
            p1 = move(pos1, d)
            s1 = score1 + p1
            if s1 > 20:
                p1wins += cnt * dcount
                continue
            for d2, d2count in universes_in_result.items():
                p2 = move(pos2, d2)
                s2 = score2 + p2
                if s2 > 20:
                    p2wins += cnt * dcount * d2count
                    continue
                nuv[(s1, p1, s2, p2)] += cnt * dcount * d2count
    universes = nuv

print("Part 2:", max([p1wins, p2wins]))

Part 1: 855624
Part 2: 187451244607486


In [56]:
# Day 22: Reactor Reboot
from itertools import product


def create_cuboid(x1, x2, y1, y2, z1, z2):
    "Creates a set with all voxels enclosed within the provided coordinates"
    return set(product(range(x1, x2 + 1), range(y1, y2 + 1), range(z1, z2 + 1)))


assert len(create_cuboid(10, 12, 10, 12, 10, 12)) == 27


def intersection(a, b):
    "Returns cuboid x1, x2, y1, y2, z1, z2 that is intersection between cuboids a and b"
    a_x1, a_x2, a_y1, a_y2, a_z1, a_z2 = a
    b_x1, b_x2, b_y1, b_y2, b_z1, b_z2 = b

    x1 = a_x1 if a_x1 > b_x1 else b_x1
    x2 = a_x2 if a_x2 < b_x2 else b_x2
    y1 = a_y1 if a_y1 > b_y1 else b_y1
    y2 = a_y2 if a_y2 < b_y2 else b_y2
    z1 = a_z1 if a_z1 > b_z1 else b_z1
    z2 = a_z2 if a_z2 < b_z2 else b_z2

    if x1 <= x2 and y1 <= y2 and z1 <= z2:
        return (x1, x2, y1, y2, z1, z2)
    return False


a = (0, 5, 0, 5, 0, 5)
b = (1, 6, 1, 6, 1, 6)
c = (1, 5, 1, 5, 1, 5)

assert intersection(a, b) == c

a = (-5, 0, -5, 0, -5, 0)
b = (-6, -1, -6, -1, -6, -1)
c = (-5, -1, -5, -1, -5, -1)

assert intersection(a, b) == c

lines = data(22)
actions = []  # action, x1, x2, y1, y2, z1, z2
for line in lines:
    action = line.split()[0] == "on", tuple(parse_ints(line.split()[1]))
    actions.append(action)

result = set()
for turn_on, coords in actions:
    # Part 1 only considers a subset of pixels
    if coords[0] > 50 or coords[0] < -50:
        continue
    cuboid = create_cuboid(*coords)
    if turn_on:
        result = result | cuboid
    else:
        result = result - cuboid
print("Part 1:", len(result))  # 648681

# Part 2
cuboids = defaultdict(int)
for turn_on, cuboid in actions:
    update = defaultdict(int)
    for existing in cuboids.copy():
        if overlapping := intersection(existing, cuboid):
            cuboids[overlapping] -= cuboids[existing]
    if turn_on:
        cuboids[cuboid] += 1

volume = 0
for coord, sign in cuboids.items():
    (x1, x2, y1, y2, z1, z2) = coord
    volume = volume + sign * (x2 - x1 + 1) * (y2 - y1 + 1) * (z2 - z1 + 1)
print("Part 2:", volume)  # 1302784472088899

Part 1: 648681
Part 2: 1302784472088899


In [3]:
# Day 23: Amphipod
# Solved by hand first. Later copied solution from
# https://github.com/mebeim/aoc/blob/master/2021/README.md#day-23---amphipod

from math import inf


def parse_rooms(fin):
    next(fin)
    next(fin)
    rooms = []

    for _ in range(2):
        l = next(fin)
        rooms.append(map("ABCD".index, (l[3], l[5], l[7], l[9])))

    return tuple(zip(*rooms))


def moves_to_room(rooms, hallway, room_size):
    for h, amphipod in enumerate(hallway):
        # Skip empty hallway slot
        if amphipod is None:
            continue

        # If the destination room contains any amphipods of a different kind we can't move
        # there
        room = rooms[amphipod]
        if any(pod != amphipod for pod in room):
            continue

        # Calculate cost of moving from hallway to its room
        cost = move_cost(room, hallway, amphipod, h, room_size, to_room=True)

        # If it's impossible to move (i.e. some other amphipod is in the way), skip it
        if cost == inf:
            continue

        # Create a new state where we move amphipod from hallway to its room. Yield new
        # state and cost to get there
        new_rooms = rooms[:amphipod] + ((amphipod,) + room,) + rooms[amphipod + 1 :]
        new_hallway = hallway[:h] + (None,) + hallway[h + 1 :]
        yield cost, (new_rooms, new_hallway)


def moves_to_hallway(rooms, hallway, room_size):
    for r, room in enumerate(rooms):
        # Room contains amphipods already in correct room
        if all(pod == r for pod in room):
            continue

        for h in range(len(hallway)):
            # Cost to move amphipod from room to hallway spot h
            cost = move_cost(room, hallway, r, h, room_size)

            if cost == inf:
                continue

            # New state where amphipod has been moved from room r to hallway h
            new_rooms = rooms[:r] + (room[1:],) + rooms[r + 1 :]
            new_hallway = hallway[:h] + (room[0],) + hallway[h + 1 :]
            yield cost, (new_rooms, new_hallway)


def possible_moves(rooms, hallway, room_size):
    try:
        yield next(moves_to_room(rooms, hallway, room_size))
    except StopIteration:
        yield from moves_to_hallway(rooms, hallway, room_size)


ROOM_DISTANCE = (
    (2, 1, 1, 3, 5, 7, 8),  # from/to room 0
    (4, 3, 1, 1, 3, 5, 6),  # from/to room 1
    (6, 5, 3, 1, 1, 3, 4),  # from/to room 2
    (8, 7, 5, 3, 1, 1, 2),  # from/to room 3
)


def move_cost(room, hallway, r, h, room_size, to_room=False):
    # h = hallway index, r = room index

    # Check if hallway path is clear
    if r + 1 < h:
        start = r + 2
        end = h + (not to_room)
    else:
        start = h + to_room
        end = r + 2
    if any(x is not None for x in hallway[start:end]):
        return inf

    # If moving to a room, the amphipod is in the hallway at spot h, otherwise it is the
    # first item in the room
    amphipod = hallway[h] if to_room else room[0]
    return 10 ** amphipod * (ROOM_DISTANCE[r][h] + (to_room + room_size - len(room)))


def done(rooms, room_size):
    for r, room in enumerate(rooms):
        if len(room) != room_size or any(pod != r for pod in room):
            return False
    return True


@lru_cache(maxsize=None)
def solve(rooms, hallway, room_size=2):
    if done(rooms, room_size):
        return 0

    best = inf

    for cost, next_state in possible_moves(rooms, hallway, room_size):
        if (new_cost := cost + solve(*next_state, room_size)) < best:
            best = new_cost

    return best


# Part 1
hallway = (None,) * 7
rooms = parse_rooms(open("2021/input-23.txt"))
print(f"Part 1: {solve(rooms, hallway)}")  # 16244
# Part 2
new_pods = [(3, 3), (2, 1), (1, 0), (0, 2)]
part2_rooms = []
for room, new in zip(rooms, new_pods):
    part2_rooms.append((room[0], *new, room[-1]))
part2_rooms = tuple(part2_rooms)
print(f"Part 2: {solve(part2_rooms, hallway, room_size=4)}")  # 43226

Part 1: 16244
Part 2: 43226


In [4]:
solve.cache_info()

CacheInfo(hits=124687, misses=133874, maxsize=None, currsize=133874)

In [58]:
# Day 23 with Dijkstra
# Couldn't find a good heuristic for A* that improved speed
from heapq import heappush, heappop
from math import inf


def a_star_search(rooms, hallway, goal, room_size):
    small_value = 0.0000000001  # Avoid comparison problems between int and None

    def heuristic():
        nonlocal small_value
        small_value += 0.0000000001
        return small_value

    moves = []
    heappush(moves, (0, (rooms, hallway)))
    cost_so_far = defaultdict(lambda: inf)
    cost_so_far[(rooms, hallway)] = 0

    while moves:
        rooms, hallway = heappop(moves)[1]

        if (rooms, hallway) == goal:
            break

        for cost, next_state in possible_moves(rooms, hallway, room_size):
            new_cost = cost_so_far[(rooms, hallway)] + cost
            if new_cost < cost_so_far[next_state]:
                cost_so_far[next_state] = new_cost
                priority = new_cost + heuristic()
                heappush(moves, (priority, next_state))

    return cost_so_far


# Part 1
hallway = (None,) * 7
rooms = parse_rooms(open("2021/input-23.txt"))
goal = (((0, 0), (1, 1), (2, 2), (3, 3)), hallway)
cost_so_far = a_star_search(rooms, hallway, goal, room_size=2)
print(f"Part 1: {cost_so_far[goal]}, explored {len(cost_so_far)} states")

# Part 2
new_pods = [(3, 3), (2, 1), (1, 0), (0, 2)]
part2_rooms = []
for room, new in zip(rooms, new_pods):
    part2_rooms.append((room[0], *new, room[-1]))
part2_rooms = tuple(part2_rooms)
goal2 = (((0, 0, 0, 0), (1, 1, 1, 1), (2, 2, 2, 2), (3, 3, 3, 3)), hallway)
cost_so_far = a_star_search(part2_rooms, hallway, goal2, room_size=4)
print(f"Part 2: {cost_so_far[goal2]}, explored {len(cost_so_far)} states")  # 43226

Part 1: 16244, explored 52713 states
Part 2: 43226, explored 79597 states


In [59]:
# Day 24: Arithmetic Logic Unit

# Find largest 14 digit number, based on instructions in input-24.txt that verifies the
# numbers individually.

# https://github.com/mebeim/aoc/blob/master/2021/README.md#day-24---arithmetic-logic-unit

from collections import deque


def find_max(constraints):
    digits = [0] * 14

    for i, j, diff in constraints:
        if diff > 0:
            digits[i], digits[j] = 9, 9 - diff
        else:
            digits[i], digits[j] = 9 + diff, 9

    # Compute the actual number from its digits.
    num = 0
    for d in digits:
        num = num * 10 + d

    return num


def find_min(constraints):
    digits = [0] * 14
    for i, j, diff in constraints:
        if diff > 0:
            digits[i], digits[j] = 1 + diff, 1
        else:
            digits[i], digits[j] = 1, 1 - diff

    # Compute the actual number from its digits.
    num = 0
    for d in digits:
        num = num * 10 + d
    return num


chunks = data(24, sep="inp w")
chunks.pop(0)  # First item is empty string

i = 0
stack = deque()
constraints = []

for chunk in chunks:
    instructions = chunk.strip().splitlines()
    if instructions[3] == "div z 1":
        value = parse_ints(instructions[14])[0]
        # print(f"Push position {i} with value {value}")
        stack.append((i, value))
    elif instructions[3] == "div z 26":
        value = parse_ints(instructions[4])[0]
        # print(f"Pop  position {i} with constraint {value}")
        j, a = stack.pop()
        # print(f"Add constraint: digit[{j}] - digit[{i}] == {a+value}")
        constraints.append((i, j, a + value))
    else:
        print("Unknown", instructions[3])
    i += 1

print("Part 1:", find_max(constraints))
print("Part 2:", find_min(constraints))

Part 1: 94992992796199
Part 2: 11931881141161


In [60]:
# Day 25: Sea Cucumber


def to_move_east(r: int) -> List[bool]:
    "Return a list with all positions in a row that shall move east."
    to_move = []
    for c, _ in enumerate(grid[r]):
        if grid[r][c] == ">" and grid[r][(c + 1) % len(grid[r])] == ".":
            to_move.append(True)
        else:
            to_move.append(False)
    return to_move


def move_all_east() -> int:
    "Move all cucumbers one step east. Returns number of moves made"
    moves = 0
    for row, _ in enumerate(grid):
        move = to_move_east(row)
        for col, _ in enumerate(grid[row]):
            if move[col]:
                moves += 1
                grid[row][(col + 1) % len(grid[row])] = grid[row][col]
                grid[row][col] = "."
    return moves


def to_move_south() -> List[bool]:
    "Return a grid with all positions that can move south"
    to_move = []
    for r, _ in enumerate(grid):
        to_move_row = []
        for c, _ in enumerate(grid[r]):
            if grid[r][c] == "v" and grid[(r + 1) % len(grid)][c] == ".":
                to_move_row.append(True)
            else:
                to_move_row.append(False)
        to_move.append(to_move_row)
    return to_move


def move_all_south():
    "Move all cucumbers one step south, returns number of moves made"
    moves = 0
    move = to_move_south()
    for row, _ in enumerate(grid):
        for col, _ in enumerate(grid[row]):
            if move[row][col]:
                moves += 1
                grid[(row + 1) % len(grid)][col] = grid[row][col]
                grid[row][col] = "."
    return moves


def print_grid():
    for row in grid:
        for char in row:
            print(char, end="")
        print()
    print()


lines = data(25)
grid = []
for line in lines:
    grid.append([char for char in line])

total_moves = 0
while True:
    moves_in_round = move_all_east()
    moves_in_round += move_all_south()
    total_moves += 1
    if moves_in_round == 0:
        break
print(total_moves)

351
